In [1]:
import config
import event
import handler
import strategy
import portfolio
import risk
import execution
import analysis as an
from datetime import datetime
import time


#SQL & API credentials. Store in config file.

access_token = config.access_token
account_id = config.account_id

sql_host = config.sql_host
sql_user = config.sql_user_analysis
sql_password = config.sql_password_analysis

In [2]:
dh = handler.DataHandler()
queue = event.EventQueue()
strategy = strategy.Strategy()
risk = risk.Risk('USDJPY', .5, 2)
portfolio = portfolio.Portfolio()
execution = execution.Execution('USD_JPY')

In [3]:
while True:

    queue.add_to_queue(dh.get_latest_rate())

    while len(queue.queue) != 0:
        next_event = queue.next_event()

        if next_event.type == 'tick':
            #Call strategy
            queue.add_to_queue(strategy.generate_signal(next_event))
            print(str(datetime.now()) + ':  ' + 'Strategy called')

        elif next_event.type == 'signal':
            # If signal requires, call risk
            if next_event.signal != 'none':
                queue.add_to_queue(risk.calculate_risk(next_event))
                print(str(datetime.now()) + ':  ' + 'Risk called')

        elif next_event.type == 'trade':
            #Call portfolio
            queue.add_to_queue(portfolio.check_order(next_event))
            print(str(datetime.now()) + ':  ' + 'Portfolio called')

        elif next_event.type == 'order':
            #Call execution
            execution.order(next_event)
            print(str(datetime.now()) + ':  ' + 'Execution called')

        elif next_event.type == 'close':
            execution.close_last_trade()
            print(str(datetime.now()) + ':  ' + 'Execution (close trade) called')
        
        elif next_event.type == 'none':
            print(str(datetime.now()) + ':  ' + 'Not passed')

        else:
            print(str(datetime.now()) + ':  ' + 'Queue Error')

    time.sleep(30)


No new signal
2017-06-09 09:34:41.402467:  Strategy called
2017-06-09 09:34:41.402501:  Not passed
No new signal
2017-06-09 09:35:13.563594:  Strategy called
2017-06-09 09:35:13.563629:  Not passed
No new signal
2017-06-09 09:35:45.904620:  Strategy called
2017-06-09 09:35:45.904654:  Not passed
No new signal
2017-06-09 09:36:18.121823:  Strategy called
2017-06-09 09:36:18.121857:  Not passed
No new signal
2017-06-09 09:36:50.515847:  Strategy called
2017-06-09 09:36:50.515881:  Not passed
No new signal
2017-06-09 09:37:22.606081:  Strategy called
2017-06-09 09:37:22.606115:  Not passed
No new signal
2017-06-09 09:37:54.865817:  Strategy called
2017-06-09 09:37:54.865852:  Not passed
No new signal
2017-06-09 09:38:27.079418:  Strategy called
2017-06-09 09:38:27.079452:  Not passed
No new signal
2017-06-09 09:38:59.246551:  Strategy called
2017-06-09 09:38:59.246585:  Not passed
No new signal
2017-06-09 09:39:31.392942:  Strategy called
2017-06-09 09:39:31.392976:  Not passed
No new sig

KeyboardInterrupt: 